# RAGAS без тестсета, т.к тест сет съел 1 лям токенов)

In [ ]:
!pip install -U -q langchain ragas gigachat pypdf wandb tiktoken faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/

In [ ]:
! pip install faithfulness

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of allennlp to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following versions that require a different python version: 0.2.0 Requires-Python ==3.6
ERROR: Could not find a version that satisfies the requirement torch<1.8.0,>=1.6.0 (from allennlp) (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1)
ERROR: No matching distribution found for torch<1.8.0,>=1.6.0


Оценак качества ретриверов на почищенных данных. Тестировать будем для RAG без памяти, для экономии токенов

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.chat_models.gigachat import GigaChat
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['CURL_CA_BUNDLE'] = ''

In [ ]:
from google.colab import userdata


Базовый пайплайн из коробки

In [ ]:
from ragas.metrics import answer_relevancy

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    # context_relevancy,
    answer_correctness,
    answer_similarity
)

In [ ]:
# from ragas.llms import LangchainLLMWrapper

In [ ]:
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper

In [ ]:
credentials_test = userdata.get('credentials_test')
llm = GigaChat(credentials=credentials_test, verify_ssl_certs=False)#auth_url = 'https://sm-auth-sd.prom-88-89-apps.ocp-geo.ocp.sigma.sbrf.ru/api/v2/oauth'

prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя.
Используй при этом только информацию из контекста. Отвечай на русском языке.
Контекст: {context}
Вопрос: {input}
Ответ:''')

In [ ]:
# credentials_test = userdata.get('credentials_test')
# llm = LangchainLLMWrapper(GigaChat(credentials=credentials_test, verify_ssl_certs=False))#auth_url = 'https://sm-auth-sd.prom-88-89-apps.ocp-geo.ocp.sigma.sbrf.ru/api/v2/oauth'

# prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя.
# Используй при этом только информацию из контекста. Отвечай на русском языке.
# Контекст: {context}
# Вопрос: {input}
# Ответ:''')
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    )

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
loader = PyPDFLoader('ai-hr-2024.pdf')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


In [ ]:
len(splits)

131

In [ ]:
# ! pip install faiss-cpu

In [ ]:

vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k" : 2})

# document_chain = create_stuff_documents_chain(
#         llm=llm,
#         prompt=prompt
#         )
# retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [ ]:
question = "Как ИИ влияет на HR процессы?"
# resp = retrieval_chain.invoke({'input': question})
# resp

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Ответьте на вопрос, основываясь только на следующем контексте. Если вы не можете ответить на вопрос с учетом контекста, ответьте, пожалуйста, ответом «не знаю»:

### CONTEXT
{context}

### QUESTION
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

from operator import itemgetter

from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

primary_qa_llm = llm

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [ ]:
result = retrieval_augmented_qa_chain.invoke({"question" : question})

In [ ]:
result

{'response': AIMessage(content='ИИ оказывает значительное влияние на HR процессы, оптимизируя и персонализируя их под конкретные потребности и характеристики сотрудников. На этапе найма ИИ помогает автоматизировать и оптимизировать процессы подбора кандидатов, сокращая время и усилия, затрачиваемые на анализ резюме, проведение интервью и принятие решений о найме. Алгоритмы машинного обучения позволяют выявить наиболее подходящих кандидатов, прогнозировать их потенциал и предсказывать успешность в будущей работе. Во время адаптации нового сотрудника ИИ может предложить персонализированные программы обучения и развития для каждого сотрудника с учетом его индивидуальных навыков, знаний и потребностей.', additional_kwargs={}, response_metadata={'token_usage': Usage(prompt_tokens=592, completion_tokens=134, total_tokens=726), 'model_name': 'GigaChat:3.1.25.3', 'finish_reason': 'stop'}, id='run-c4f8c0b1-6f45-4a5d-9e08-f5d2af1e0f5e-0'),
 'context': [Document(metadata={'source': 'ai-hr-2024.pd

1400 токенов

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [ ]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()

In [ ]:
question_generation_llm = GigaChat(credentials=credentials_test, verify_ssl_certs=False)

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

In [ ]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
You are a professional document analyst. For each context, create a context-specific question. Avoid creating generic or general questions. Do everything in Russian.

question: a question about the context.

Format the output as JSON with the following keys:
question

context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=docs[0],
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})
output_dict = question_output_parser.parse(response.content)

In [ ]:
for k, v in output_dict.items():
  print(k)
  print(v)

question
Какой месяц указан на странице?


In [ ]:
! pip install -q -U tqdm

In [ ]:
from tqdm import tqdm

qac_triples = []

for text in tqdm(docs[:10]):
  messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
  )
  response = question_generation_chain.invoke({"content" : messages})
  try:
    output_dict = question_output_parser.parse(response.content)
  except Exception as e:
    continue
  output_dict["context"] = text
  qac_triples.append(output_dict)

100%|██████████| 10/10 [00:10<00:00,  1.08s/it]


In [ ]:
qac_triples[5]

{'question': 'Какие основные задачи искусственного интеллекта заключаются в усилении существующих качеств и возможностей продуктов? Приведите примеры таких задач.',
 'context': Document(metadata={'source': 'ai-hr-2024.pdf', 'page': 5}, page_content='2023\n2020\n2015\n2010\nКоллаборативность\nИИ\nУдобство / Мобильность\nФункциональность\nСегодня основная задача ИИ заключается \nв усилении существующих качеств \nи возможностей продуктов. Например, \nон может упрощать совместную работу \nнескольких пользователей с помощью \nнапоминаний и подсказок, улучшать \nаналитические возможности за счет обработки \nданных и расширять функциональность \nпродукта с применением генеративных \nмоделей.\nЭффективное применение ИИ предполагает \nпреумножение возможностей продукта. \nЕсли же он применяется в отрыве \nот качественного фундамента, продукт \nстановится неактуальным и быстро теряет \nвнимание пользователей. Первый год активного \nиспользования генеративных моделей показал, \nчто выживают те ко

In [ ]:

answer_generation_llm = GigaChat(credentials=credentials_test, verify_ssl_certs=False)
answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\

You are a professional document analyst. For each question and context, create an answer. Do everything in Russian language.

answer: a answer about the context.

Format the output as JSON with the following keys:
answer

question: {question}
context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["question"],
    format_instructions=format_instructions
)

answer_generation_chain = bare_template | answer_generation_llm

response = answer_generation_chain.invoke({"content" : messages})
output_dict = answer_output_parser.parse(response.content)

In [ ]:
for k, v in output_dict.items():
  print(k)
  print(v)

answer
август


In [ ]:
for triple in tqdm(qac_triples):
  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["question"],
      format_instructions=format_instructions
  )
  response = answer_generation_chain.invoke({"content" : messages})
  try:
    output_dict = answer_output_parser.parse(response.content)
  except Exception as e:
    continue
  triple["answer"] = output_dict["answer"]

100%|██████████| 10/10 [00:23<00:00,  2.39s/it]


In [ ]:
! pip install -q -U datasets

In [ ]:
import pandas as pd
from datasets import Dataset

ground_truth_qac_set = pd.DataFrame(qac_triples)


In [ ]:
ground_truth_qac_set

,question,context,answer
0,Какой месяц указан на странице?,page_content='Август 2024' metadata={'source':...,август
1,Какие ключевые барьеры внедрения искусственног...,page_content='В первом выпуске совместного исс...,Ключевые барьеры внедрения искусственного инте...
2,Какие задачи в области управления человеческим...,page_content='Искусственный интеллект –одно из...,Автоматизация задач в области управления челов...
3,Какие риски связаны с безопасностью данных в к...,"page_content='1. Риски, связанные с безопаснос...","Риски, связанные с безопасностью данных, включ..."
4,Какие цели развития искусственного интеллекта?,page_content='Технологии Доверия\nИскусственны...,Целью развития искусственного интеллекта являе...
5,Какие основные задачи искусственного интеллект...,page_content='2023\n2020\n2015\n2010\nКоллабор...,Основные задачи искусственного интеллекта закл...
6,Какой запрос специалист может отправить чат-боту?,page_content='Технологии Доверия\nЧат-бот\nакт...,Специалист может отправить чат-боту запрос на ...
7,Какое качество продукта является ключевым?,"page_content='Такой опыт подсказывает, что клю...",способность решать поставленную задачу
8,Какие задачи могут быть решены с использование...,page_content='Технологии Доверия\nПрименение И...,С использованием технологий искусственного инт...
9,Какие преимущества предоставляет использование...,page_content='Технологии Доверия\nВнедрение ис...,Использование искусственного интеллекта в упра...


In [ ]:

ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "reference"})


In [ ]:


eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [ ]:
eval_dataset

Dataset({
    features: ['question', 'context', 'reference'],
    num_rows: 10
})

Как мы можем заметить половина вопросов мусорные

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    # context_relevancy,
    answer_correctness,
    answer_similarity
)

# from ragas.metrics.critique import harmfulness
from ragas import evaluate

def create_ragas_dataset(rag_pipeline, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_pipeline.invoke({"question" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer["response"].content,
         "contexts" : [context.page_content for context in answer["context"]],
         "reference" : [row["reference"]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        # context_precision,
        faithfulness,
        answer_relevancy,
        # context_recall,
        # context_relevancy,
        answer_correctness,
        answer_similarity
    ],
    llm=llm,
    embeddings=embedding
  )
  return result

In [ ]:
basic_qa_ragas_dataset = create_ragas_dataset(retrieval_augmented_qa_chain, eval_dataset)

100%|██████████| 10/10 [00:15<00:00,  1.60s/it]


In [ ]:
q = basic_qa_ragas_dataset.to_pandas()

In [ ]:
# ref = q['ground_truths']

In [ ]:
li = q.copy()

In [ ]:
q = li.copy()

In [ ]:
q['reference'] = q['reference'].apply(lambda x: x[0])

In [ ]:
q

,question,answer,contexts,reference
0,Какой месяц указан на странице?,Август,"[Август 2024, какие то проблемы со здоровьем, ...",август
1,Какие ключевые барьеры внедрения искусственног...,Не знаю,"[внедрение ИИ, но пока оно не достигло HR -\nп...",Ключевые барьеры внедрения искусственного инте...
2,Какие задачи в области управления человеческим...,Задачи в области управления человеческим капит...,[инструмент для принятия стратегических решени...,Автоматизация задач в области управления челов...
3,Какие риски связаны с безопасностью данных в к...,"В контексте разработки стратегии риски, связан...","[1. Риски, связанные с безопасностью данных \n...","Риски, связанные с безопасностью данных, включ..."
4,Какие цели развития искусственного интеллекта?,Целями развития искусственного интеллекта явля...,[Искусственный интеллект –одно из самых иннова...,Целью развития искусственного интеллекта являе...
5,Какие основные задачи искусственного интеллект...,Основные задачи искусственного интеллекта закл...,[2023\n2020\n2015\n2010\nКоллаборативность\nИИ...,Основные задачи искусственного интеллекта закл...
6,Какой запрос специалист может отправить чат-боту?,Специалист может отправить чат-боту запрос на ...,[Технологии Доверия\nЧат-бот\nактивируется\nМо...,Специалист может отправить чат-боту запрос на ...
7,Какое качество продукта является ключевым?,Ключевым качеством продукта является его спосо...,"[Такой опыт подсказывает, что ключевым качеств...",способность решать поставленную задачу
8,Какие задачи могут быть решены с использование...,Технологии искусственного интеллекта могут быт...,[В стратегии мы описали использование искусств...,С использованием технологий искусственного инт...
9,Какие преимущества предоставляет использование...,Использование искусственного интеллекта в упра...,[персонализированные рекомендации \nпо развити...,Использование искусственного интеллекта в упра...


In [ ]:
# q['question'] = q['question'].apply(lambda x: [x])


In [ ]:
# q['answer'] = q['answer'].apply(lambda x: [x])

In [ ]:
# q['reference'] = ref

In [ ]:
basic_qa_ragas_dataset = Dataset.from_pandas(q)

In [ ]:
basic_qa_ragas_dataset[0]

{'question': 'Какой месяц указан на странице?',
 'answer': 'Август',
 'contexts': ['Август 2024',
  'какие то проблемы со здоровьем, тогда сообщить, что есть возможность либо получить налоговый вычет, либо \nкомпенсацию на расходы на приобретение дорогостоящих медикаментов. Цель –донести максимально \nполезную информацию до конечного сотрудника. Сейчас все это является предметом управления некоторого \nколичества менеджеров, и здесь может быть потенциал, чтобы через ИИ -механизмы делать это более \nтаргетированно и в релевантных каналах.\nЕще, как вы помните с прошлогоднего интервью, у нас используется ИИ для прогнозирования увольнений \nс 2018 года с целью удержания сотрудников и минимизации затрат компании при их увольнении, \nпредсказывается порядка 80% увольнений. Также ИИ проводит exit -интервью с помощью синтезированного \nголоса и проводит аналитику причин увольнения. ИИ с 2020 года был встроен и в чат -бот-помощник, он может \nгенерировать уникальные ответы подключенных баз и с

In [ ]:
basic_qa_ragas_dataset[0]

{'question': 'Какой месяц указан на странице?',
 'answer': 'Август',
 'contexts': ['Август 2024',
  'какие то проблемы со здоровьем, тогда сообщить, что есть возможность либо получить налоговый вычет, либо \nкомпенсацию на расходы на приобретение дорогостоящих медикаментов. Цель –донести максимально \nполезную информацию до конечного сотрудника. Сейчас все это является предметом управления некоторого \nколичества менеджеров, и здесь может быть потенциал, чтобы через ИИ -механизмы делать это более \nтаргетированно и в релевантных каналах.\nЕще, как вы помните с прошлогоднего интервью, у нас используется ИИ для прогнозирования увольнений \nс 2018 года с целью удержания сотрудников и минимизации затрат компании при их увольнении, \nпредсказывается порядка 80% увольнений. Также ИИ проводит exit -интервью с помощью синтезированного \nголоса и проводит аналитику причин увольнения. ИИ с 2020 года был встроен и в чат -бот-помощник, он может \nгенерировать уникальные ответы подключенных баз и с

In [ ]:
basic_qa_ragas_dataset.to_csv("basic_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

45445

In [ ]:
basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

KeyboardInterrupt: 